In [16]:

import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2 

DataDir="F:/2022_Summer/data"
Categories=["Post-Injury", "Pre-Injury"]

trainingdata=[] 
def createtrainingdata():
#creates a data set of images and labels 
    print("Training data is loading...")
    premat=[]
    postmat=[]
    for category in Categories: 
        path=os.path.join(DataDir,category)
        classnum=Categories.index(category)
        for vid in os.listdir(path):
            vpath=os.path.join(path,vid)
            vid_array=[]
            for frame in os.listdir(vpath): 
                img_array=cv2.imread(os.path.join(vpath,frame),0)
                
                if classnum ==0:
                    postmat.append(img_array)
                else:
                    premat.append(img_array)
            trainingdata.append([vid_array,classnum])
    
    postav=np.average(np.array(postmat))
    print(postav)
    preav=np.average(np.array(premat))
    print(preav)
    
    #Centers the data set to control for the lighting
    print("Training data is being centered...")
    for d in trainingdata: 
        if d[1]==1:
            d[0]=d[0]-preav
        else:
            d[0]=d[0]-postav
                           
    print("Done!")
createtrainingdata()

Training data is loading...
80.16980072784314
80.84310740398791
Training data is being centered...
Done!


In [4]:
#Randomizes the order of the data set 
import random 
random.shuffle(trainingdata)
print("Data is randomized!")

Data is randomized!


In [10]:
#Changes the trainingdata into X and Y numpy arrays to be inputted into the model 
X=[]
Y=[]
for features,label in trainingdata:
    X.append(features)
    Y.append(label)

X=np.array(X)
Y=np.array(Y)
X=X.reshape(-1,X[1].shape[0],X[1].shape[1],1)

print("Data is formatted!")

Data is formatted!


In [11]:
#saves training data 
import pickle 
pickle_out=open("X.pickle", "wb")
pickle.dump(X,pickle_out)
pickle_out.close()
pickle_out=open("Y.pickle", "wb")
pickle.dump(Y,pickle_out)
pickle_out.close()
print("Data is saved!")

Data is saved!


In [12]:

import tensorflow as tf 
import pickle 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time 

NAME="Pre-vs-Post-Injury-CNN-{}".format(int(time.time()))

tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

X=pickle.load(open("X.pickle", "rb"))
Y=pickle.load(open("Y.pickle", "rb"))
#Normalizes the values 
X=X/255.0

model=Sequential()
model.add(Conv2D(64, (3,3),input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(1))

model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

model.fit(X,Y, batch_size=32,epochs=3,validation_split=0.1,callbacks=[tensorboard])


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.